In [10]:
import preprocessing as pp
import pandas as pd
import numpy as np
import time 
# import xpress as xp # FICO Xprerss Solver
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from k_means_constrained import KMeansConstrained
# xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

In [11]:
customer_df, candidate_df, supplier_df, vehicle_df, distance_w_to_s_df, distance_w_to_c_df, demand_cus_period_df, demand_cus_period_scene_df = pp.read_and_prep_data()

# 2 methods for clustering

customer_df, cluster_center_df = pp.ori_cluster_by_cus_loc(customer_df, n_clusters=400, random_state=42, n_init=10)
# customer_df, cluster_center_df = pp.const_cluster_by_cus_loc(customer_df, n_clusters=100, size_min=4, size_max=400, random_state=42)

agg_dem_cus_period_df = pp.agg_dem_cus_period(demand_cus_period_df,customer_df)

agg_dem_cus_period_scene_df = pp.agg_dem_cus_period_scene(demand_cus_period_scene_df, customer_df)

In [12]:
agg_dem_cus_period_df

0        1        2        3        4        5  \
Cluster ProductIndex                                                        
0       0               13.0     27.0     39.0     46.0     68.0     75.0   
        1               16.0     33.0     49.0     80.0     96.0    100.0   
        2              111.0    174.0    315.0    402.0    500.0    627.0   
        3               12.0     31.0     36.0     50.0     70.0     77.0   
1       0              314.0    543.0    855.0    971.0   1436.0   1703.0   
...                      ...      ...      ...      ...      ...      ...   
398     3               82.0    206.0    230.0    389.0    455.0    577.0   
399     0             1697.0   3952.0   5649.0   6812.0   7583.0   9312.0   
        1             1377.0   2590.0   4818.0   6828.0   8098.0   8411.0   
        2             7640.0  14654.0  19938.0  28589.0  41972.0  47586.0   
        3              593.0   1294.0   2241.0   2448.0   3651.0   4227.0   

                            6        7        8        9  
Cluster ProductIndex                                      
0       0                94.0    102.0    116.0    126.0  
        1               135.0    145.0    163.0    179.0  
        2               778.0    855.0    953.0   1055.0  
        3                89.0    104.0    124.0    137.0  
1       0              1935.0   2387.0   2551.0   2816.0  
...                       ...      ...      ...      ...  
398     3               580.0    694.0    824.0    883.0  
399     0             11303.0  13681.0  15112.0  16957.0  
        1              9696.0  12116.0  13859.0  15053.0  
        2             51140.0  62184.0  71695.0  78937.0  
        3              4389.0   5415.0   6317.0   6762.0  

[1600 rows x 10 columns]

In [13]:
agg_dem=agg_dem_cus_period_scene_df.reset_index()

In [14]:
agg_dem.columns

Index([     'Cluster', 'ProductIndex',  'PeriodIndex',              0,
                    1,              2,              3,              4,
                    5,              6,
       ...
                   90,             91,             92,             93,
                   94,             95,             96,             97,
                   98,             99],
      dtype='object', length=103)

In [ ]:
agg_dem = agg_dem.drop(columns=['Cluster', 'ProductIndex', 'PeriodIndex'])

KeyError: "['Cluster', 'ProductIndex', 'PeriodIndex'] not found in axis"

In [16]:
agg_dem = agg_dem.transpose()

In [17]:
def const_cluster_by_cus_loc(customer_df:pd.DataFrame, n_clusters:int=100, size_min:int=4, size_max:int=400, random_state:int=42)->tuple:
    df = customer_df.copy()

    kmeans_constrained = KMeansConstrained(
                                            n_clusters=n_clusters,
                                            size_min=size_min,
                                            size_max=size_max,
                                            random_state=random_state
                                        )

    df['Cluster'] = kmeans_constrained.fit_predict(customer_df[['CustomerEasting','CustomerNorthing','CustomerPopulation']])

    cluster_center_df = pd.DataFrame(kmeans_constrained.cluster_centers_)
    cluster_center_df.drop(2,axis = 1, inplace = True)
    cluster_center_df.reset_index(inplace=True)
    cluster_center_df.rename(columns = {'index':'Cluster', 0:'CustomerEasting', 1:'CustomerNorthing'}, inplace = True)
    cluster_center_df.set_index('Cluster',inplace=True)

    return df, cluster_center_df


In [18]:
agg_dem

,0,1,2,3,4,5,6,7,8,9,...,15990,15991,15992,15993,15994,15995,15996,15997,15998,15999
0,12.0,35.0,48.0,71.0,103.0,108.0,147.0,191.0,155.0,208.0,...,524.0,1262.0,3096.0,3165.0,5685.0,5437.0,7863.0,8854.0,10714.0,12368.0
1,12.0,30.0,35.0,47.0,70.0,79.0,105.0,82.0,90.0,133.0,...,628.0,1316.0,2076.0,2736.0,2866.0,4535.0,4808.0,4837.0,5654.0,5954.0
2,11.0,21.0,34.0,40.0,62.0,74.0,77.0,80.0,104.0,81.0,...,588.0,1468.0,1789.0,1893.0,2701.0,2963.0,3167.0,4896.0,4629.0,5093.0
3,13.0,27.0,49.0,48.0,87.0,79.0,129.0,144.0,160.0,177.0,...,683.0,1306.0,2716.0,2801.0,5031.0,5552.0,5626.0,8392.0,7057.0,8114.0
4,13.0,33.0,39.0,55.0,81.0,89.0,88.0,126.0,146.0,157.0,...,530.0,1220.0,2226.0,2201.0,4108.0,4139.0,5866.0,6962.0,6113.0,8460.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,13.0,23.0,26.0,24.0,34.0,36.0,36.0,30.0,18.0,9.0,...,554.0,1011.0,1574.0,1437.0,1888.0,1540.0,1544.0,1204.0,1162.0,433.0
96,12.0,28.0,54.0,52.0,101.0,98.0,118.0,155.0,227.0,248.0,...,645.0,1420.0,2628.0,3436.0,5318.0,6388.0,5990.0,7674.0,9570.0,11996.0
97,13.0,25.0,48.0,55.0,66.0,83.0,116.0,105.0,128.0,124.0,...,584.0,1428.0,2655.0,2469.0,3732.0,3852.0,4274.0,5609.0,6254.0,8108.0
98,13.0,22.0,37.0,42.0,61.0,60.0,67.0,81.0,91.0,86.0,...,648.0,1041.0,1774.0,2235.0,2861.0,3432.0,3966.0,3299.0,4776.0,4773.0


In [ ]:
agg_dem_cus_period_df.max()

In [ ]:
# def create_dis_mat_df(depart_df:pd.DataFrame, arrive_df:pd.DataFrame, method: 'cityblock'):
#     # Assuming warehouse_df and customer_df have columns ['x', 'y']
#     depart_coords = depart_df[['CandidateEasting', 'CandidateNorthing']].values
#     arrive_coords = arrive_df[['CustomerEasting', 'CustomerNorthing']].values

#     # Compute Euclidean distance matrix
#     distance_matrix = cdist(depart_coords, arrive_coords, metric=method)/1000
#     # display(distance_matrix)
#     # Convert to a Pandas DataFrame for better readability
#     distance_df = pd.DataFrame(distance_matrix, 
#                             index=depart_df.index, 
#                             columns=arrive_df.index)

#     return distance_df

# def calculate_cost_from_w_to_cluster(distance_df:pd.DataFrame, vehicle_df:pd.DataFrame):
#     cost_w_to_cluster = distance_df.copy()
#     for w in range(len(distance_df.index)):
#         for c in range(len(distance_df.columns)):
#             cost_w_to_cluster.at[w,c] = 2 * distance_df.at[w,c] * vehicle_df.at[2,'VehicleCostPerMileAndTonneOverall'] / 1000   
#     return cost_w_to_cluster

# def calculate_cost_from_w_to_s(distance_df:pd.DataFrame, vehicle_df:pd.DataFrame, supplier_df:pd.DataFrame):
#     cost_w_to_s = distance_df.copy()
#     for w in range(len(distance_df.index)):
#         for s in range(len(distance_df.columns)):
#             if supplier_df.at[s,'SupplierVehicleType'] == 0:
#                 cost_w_to_s.at[w,s] = 2 * distance_df.at[w,s] * vehicle_df.at[0,'VehicleCostPerMileAndTonneOverall'] / 1000   
#             elif supplier_df.at[s,'SupplierVehicleType'] == 1:
#                 cost_w_to_s.at[w,s] = 2 * distance_df.at[w,s] * vehicle_df.at[1,'VehicleCostPerMileAndTonneOverall'] / 1000   
#             else:
#                 pass
#     return cost_w_to_s

In [ ]:
distance_w_to_cluster_df = pp.create_dis_mat_df(candidate_df, cluster_center_df,'cityblock')

# Create Cost
cost_w_to_cluster = pp.calculate_cost_from_w_to_cluster(distance_w_to_cluster_df, vehicle_df)
cost_w_to_s = pp.calculate_cost_from_w_to_s(distance_w_to_s_df, vehicle_df, supplier_df)

In [ ]:
# # Create the model
# model = xp.problem(name = 'MEWLP')
# # Sets and Notation
# S = list(supplier_df.index) # Supplier Index
# S_P0 = list(supplier_df[supplier_df['SupplierProductGroup'] == 0]['SupplierProductGroup'].index)
# S_P1 = list(supplier_df[supplier_df['SupplierProductGroup'] == 1]['SupplierProductGroup'].index)
# S_P2 = list(supplier_df[supplier_df['SupplierProductGroup'] == 2]['SupplierProductGroup'].index)
# S_P3 = list(supplier_df[supplier_df['SupplierProductGroup'] == 3]['SupplierProductGroup'].index)
# S_P_dict = {0:S_P0, 1:S_P1, 2:S_P2, 3:S_P3}

# W = list(candidate_df.index) # Warehouse Index
# C = list(cluster_center_df.index) # Cluster Index
# P = list(agg_dem_cus_period_df.reset_index()['ProductIndex'].unique())
# T = list(list(agg_dem_cus_period_df.columns))

# # Output Variables
# x = np.array([xp.var(f'x_{w}_{c}_{p}_{t}', vartype = xp.binary) for w in W for c in C for p in P for t in T], dtype = xp.npvar).reshape(len(W), len(C), len(P), len(T))
# y = np.array([xp.var(f'y_{w}_{t}', vartype = xp.binary) for w in W for t in T], dtype = xp.npvar).reshape(len(W), len(T))
# o = np.array([xp.var(f'o_{w}', vartype = xp.binary) for w in W], dtype = xp.npvar).reshape(len(W))
# v = np.array([xp.var(f'v_{w}_{c}_{p}_{t}', vartype = xp.continuous, lb = 0) for w in W for c in C for p in P for t in T], dtype = xp.npvar).reshape(len(W), len(C), len(P), len(T))
# z = np.array([xp.var(f'z_{w}_{s}_{t}', vartype = xp.continuous, lb = 0) for w in W for s in S for t in T], dtype = xp.npvar).reshape(len(W), len(S), len(T))

# model.addVariable(x, y, o, v, z)

# # Constraints 
# for w in W:
#     Capacity_W = candidate_df.loc[w,'Capacity']
#     for t in T:
#         model.addConstraint(xp.constraint(o[w] >= y[w,t]))
#         model.addConstraint(xp.constraint(xp.Sum(v[w,c,p,t] for c in C for p in P) <= Capacity_W * y[w,t]))
#         if t != 0:
#             model.addConstraint(xp.constraint(y[w, t] >= y[w,t-1]))    

#         for p in P:
#             model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for s in S_P_dict[p]) == xp.Sum(v[w,c,p,t] for c in C)))
#             for c in C:
#                 model.addConstraint(xp.constraint(x[w,c,p,t] <= y[w,t]))  

# for t in T:
#     for c in C:
#         for p in P:
#             Demand = agg_dem_cus_period_df.loc[(c,p),t]
#             model.addConstraint(xp.constraint(xp.Sum(x[w,c,p,t] for w in W) == 1))

#             for w in W:
#                 model.addConstraint(xp.constraint(v[w,c,p,t] == Demand * x[w,c,p,t]))    

# for s in S:
#     Capacity_S = supplier_df.loc[s,'SupplierCapacity']
#     # Product_S = supplier_df.loc[s,'SupplierProductGroup']
#     for t in T:
#         model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for w in W) <= Capacity_S))



# Setup_cost = xp.Sum(candidate_df.loc[w,'Setup'] * o[w] for w in W)
# Operating_cost = xp.Sum(candidate_df.loc[w,'Operating'] * y[w,t] for w in W for t in T)
# Tra_w_s_cost = xp.Sum(cost_w_to_s.loc[w,s] * z[w,s,t] for w in W for s in S for t in T)
# Tra_w_c_cost = xp.Sum(cost_w_to_cluster.loc[w,c] * v[w,c,p,t] for w in W for c in C for p in P for t in T)

# obj = Setup_cost + Operating_cost + Tra_w_s_cost + Tra_w_c_cost

# model.setObjective(obj, sense = xp.minimize)
# model.setControl('miprelstop', 1e-3)
# model.setControl('maxtime', 600)
# tic_time = time.time()
# # Solve the problem
# model.solve()
# toc_time = time.time()
# solve_time = toc_time - tic_time
# obj_value = model.getObjVal()

In [ ]:
asdfasd

In [ ]:
candidate_df

In [ ]:
x_matrix = model.getSolution(x)
y_matrix = model.getSolution(y)
v_matrix = model.getSolution(v)
z_matrix = model.getSolution(z)

In [ ]:
w_open_output = pd.DataFrame(y_matrix).astype(int)

In [ ]:
Capacity_S = supplier_df.loc[0,'SupplierCapacity']

In [ ]:
s_flow_output = pd.DataFrame()
for s in S:
    Capacity_S = supplier_df.loc[s,'SupplierCapacity']
    if np.any((Capacity_S - pd.DataFrame(z_matrix[:,s,:,:].sum(axis = 0)).sum(axis = 0)) < 0) :
        raise ValueError(f'Supplier {s} exceed its capacity.')
    else:
        for w in W:
            sub_df = pd.DataFrame(z_matrix[w,s,:,:])
            sub_df.reset_index(inplace= True)
            sub_df.rename(columns = {'index':'ProductIndex'},inplace = True)
            sub_df['CandidateIndex'] = w
            sub_df['SupplierIndex'] = s

            s_flow_output = pd.concat([s_flow_output, sub_df])            

s_flow_output = s_flow_output.groupby(['SupplierIndex','CandidateIndex','ProductIndex']).sum()
    

In [ ]:
Capacity_S - pd.DataFrame(z_matrix[:,s,:,:].sum(axis = 0)).sum(axis = 0)

In [ ]:
w_flow_output = pd.DataFrame()
for w in W:
    Capacity_W = candidate_df.loc[w,'Capacity']
    if np.any((Capacity_W - pd.DataFrame(v_matrix[w,:,:,:].sum(axis = 0)).sum(axis = 0)) < 0) :
        raise ValueError(f'Warehose {w} exceed its capacity.')
    else:
        for c in C:
            sub_df = pd.DataFrame(v_matrix[w,c,:,:])
            sub_df.reset_index(inplace= True)
            sub_df.rename(columns = {'index':'ProductIndex'},inplace = True)
            sub_df['CandidateIndex'] = w
            sub_df['ClusterIndex'] = c

            w_flow_output = pd.concat([w_flow_output, sub_df])

w_flop_output = w_flow_output.groupby(['CandidateIndex','ClusterIndex','ProductIndex']).sum()

In [ ]:
Capacity_W - w_flop_output.loc[(0),:].sum(axis = 0) == Capacity_W - pd.DataFrame(v_matrix[0,:,:,:].sum(axis = 0)).sum(axis = 0) 

In [ ]:
np.any((Capacity_W - pd.DataFrame(v_matrix[0,:,:,:].sum(axis = 0)).sum(axis = 0)) < 0)

In [ ]:
Capacity_W = candidate_df.loc[0,'Capacity']

In [ ]:
Capacity_W

In [ ]:
v_matrix[0,:,:,:].sum(axis = 1)

In [ ]:
pd.DataFrame(v_matrix[0,:,:,:].sum(axis = 1))

In [ ]:
cover_output = agg_dem_cus_period_df.copy()
cover_output.loc[:,:] = -1
for c in C:
    for p in P:
        for t in T:
            cover_from_w = np.where(x_matrix[:,c,p,t] != 0)[0]
            cover_from_w_check = np.where(v_matrix[:,c,p,t] != 0)[0]
            if np.size(cover_from_w) == 1 and cover_from_w == cover_from_w_check: # Check that only one w will c for p during t 
                cover_output.at[(c,p),t] = int(cover_from_w[0])
            else:
                raise ValueError(f"More Than 2 warehouse covers customer {c} for product {p} during time {t}")

cover_output = cover_output.astype('int')  

if np.any(cover_output == -1):
    raise ValueError('Some entry of this df has not filled in yet.')

In [ ]:
cover_output.head(10)

In [ ]:
np.size(np.where(x_matrix[:,0,1,2] != 0)[0])

In [ ]:
np.where(v_matrix[:,0,1,2] != 0)[0]

In [ ]:
v_matrix[:,0,1,2]

In [ ]:
agg_dem_cus_period_df

In [ ]:
v[0,0,0,0]

In [ ]:
supplier_df

In [ ]:
asdfasdf

In [ ]:
# Create the model
model2 = xp.problem(name = 'MEWLP2')
# Sets and Notation
S = list(supplier_df.index) # Supplier Index
S_P0 = list(supplier_df[supplier_df['SupplierProductGroup'] == 0]['SupplierProductGroup'].index)
S_P1 = list(supplier_df[supplier_df['SupplierProductGroup'] == 1]['SupplierProductGroup'].index)
S_P2 = list(supplier_df[supplier_df['SupplierProductGroup'] == 2]['SupplierProductGroup'].index)
S_P3 = list(supplier_df[supplier_df['SupplierProductGroup'] == 3]['SupplierProductGroup'].index)
S_P_dict = {0:S_P0, 1:S_P1, 2:S_P2, 3:S_P3}

W = list(candidate_df.index) # Warehouse Index
C = list(cluster_center_df.index) # Cluster Index
P = list(agg_dem_cus_period_df.reset_index()['ProductIndex'].unique())
T = list(list(agg_dem_cus_period_df.columns))

# Output Variables
x = np.array([xp.var(f'x_{w}_{c}_{t}', vartype = xp.binary) for w in W for c in C for t in T], dtype = xp.npvar).reshape(len(W), len(C), len(T))
y = np.array([xp.var(f'y_{w}_{t}', vartype = xp.binary) for w in W for t in T], dtype = xp.npvar).reshape(len(W), len(T))
o = np.array([xp.var(f'o_{w}', vartype = xp.binary) for w in W], dtype = xp.npvar).reshape(len(W))
v = np.array([xp.var(f'v_{w}_{c}_{p}_{t}', vartype = xp.continuous, lb = 0) for w in W for c in C for p in P for t in T], dtype = xp.npvar).reshape(len(W), len(C), len(P), len(T))
z = np.array([xp.var(f'z_{w}_{s}_{t}', vartype = xp.continuous, lb = 0) for w in W for s in S for t in T], dtype = xp.npvar).reshape(len(W), len(S), len(T))

model2.addVariable(x, y, o, v, z)

# Constraints 
for w in W:
    Capacity_W = candidate_df.loc[w,'Capacity']
    for t in T:
        model2.addConstraint(xp.constraint(o[w] >= y[w,t]))
        model2.addConstraint(xp.constraint(xp.Sum(v[w,c,p,t] for c in C for p in P) <= Capacity_W * y[w,t]))
        if t != 0:
            model2.addConstraint(xp.constraint(y[w, t] >= y[w,t-1]))    

        for p in P:
            model2.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for s in S_P_dict[p]) == xp.Sum(v[w,c,p,t] for c in C)))
        for c in C:
            model2.addConstraint(xp.constraint(x[w,c,t] <= y[w,t]))  

for t in T:
    for c in C:
        model2.addConstraint(xp.constraint(xp.Sum(x[w,c,t] for w in W) == 1))
        # model2.addConstraint(xp.constraint(xp.Sum(x[w,c,t] for w in W) >= 1))
        for p in P:
            Demand = agg_dem_cus_period_df.loc[(c,p),t]
            # model2.addConstraint(xp.constraint(xp.Sum(v[w,c,p,t] for w in W) >= Demand))
            for w in W:
                model2.addConstraint(xp.constraint(v[w,c,p,t] == Demand * x[w,c,t]))    
                # model2.addConstraint(xp.constraint(v[w,c,p,t] <= Demand * x[w,c,t]))    

for s in S:
    Capacity_S = supplier_df.loc[s,'SupplierCapacity']
    # Product_S = supplier_df.loc[s,'SupplierProductGroup']
    for t in T:
        model2.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for w in W) <= Capacity_S))



Setup_cost = xp.Sum(candidate_df.loc[w,'Setup'] * o[w] for w in W)
Operating_cost = xp.Sum(candidate_df.loc[w,'Operating'] * y[w,t] for w in W for t in T)
Tra_w_s_cost = xp.Sum(cost_w_to_s.loc[w,s] * z[w,s,t] for w in W for s in S for t in T)
Tra_w_c_cost = xp.Sum(cost_w_to_cluster.loc[w,c] * v[w,c,p,t] for w in W for c in C for p in P for t in T)

obj = Setup_cost + Operating_cost + Tra_w_s_cost + Tra_w_c_cost

model2.setObjective(obj, sense = xp.minimize)
model2.setControl('miprelstop', 1e-3)
model2.setControl('maxtime', 3600)
tic_time = time.time()
# Solve the problem
model2.solve()
toc_time = time.time()
solve_time = toc_time - tic_time
obj_value = model2.getObjVal()

In [ ]:
x_matrix = model2.getSolution(x)
y_matrix = model2.getSolution(y)
v_matrix = model2.getSolution(v)
z_matrix = model2.getSolution(z)

In [ ]:
# model2.solve()
# toc_time = time.time()
# solve_time = toc_time - tic_time
# obj_value = model2.getObjVal()